In [17]:
import pandas as pd
from pathlib import Path
import AppHelpers as ah

In [18]:
# load citibike data for each month
jan_data_path = Path("Data/JC-202001-citibike-tripdata.csv")
feb_data_path = Path("Data/JC-202002-citibike-tripdata.csv")
mar_data_path = Path("Data/JC-202003-citibike-tripdata.csv")
apr_data_path = Path("Data/JC-202004-citibike-tripdata.csv")
may_data_path = Path("Data/JC-202005-citibike-tripdata.csv")

jan_data = pd.read_csv(jan_data_path)
feb_data = pd.read_csv(feb_data_path)
mar_data = pd.read_csv(mar_data_path)
apr_data = pd.read_csv(apr_data_path)
may_data = pd.read_csv(may_data_path)

# Combine all data into a single DataFrame
combined_data = pd.concat([feb_data, mar_data, apr_data, may_data], ignore_index=True)

# load citibike stations data
stations_data = pd.read_csv("Resources/citibike_stations.csv")

# add month column as first column and name it based on the month in the starttime column
combined_data.insert(0, "Month", combined_data['starttime'].str[5:7])
# convert month column value to full month name
combined_data['Month'] = combined_data['Month'].replace(['02', '03', '04', '05'], ['Feb', 'Mar', 'Apr', 'May'])

# add start station and end station city and zipcode columns
combined_data.insert(1, "start_station_city","")
combined_data.insert(2, "start_station_zipcode","")
combined_data.insert(3, "end_station_city","")
combined_data.insert(4, "end_station_zipcode","")

# combined_data.head()


In [ ]:
# slice the dataframe to get the first 100 rows
# combined_data = combined_data.iloc[0:50]
print(len(combined_data))

# populate the start station city and zipcode and end station city and zipcode columns with the values from stations_data based on the station id
for index, row in combined_data.iterrows():
    start_station_id = row["start station id"]
    end_station_id = row["end station id"]
    start_station = stations_data.loc[stations_data["station_id"] == start_station_id]
    end_station = stations_data.loc[stations_data["station_id"] == end_station_id]
    combined_data.loc[index, "start_station_city"] = start_station["city"].values[0]
    combined_data.loc[index, "start_station_zipcode"] = start_station["zipcode"].values[0]
    combined_data.loc[index, "end_station_city"] = end_station["city"].values[0]
    combined_data.loc[index, "end_station_zipcode"] = end_station["zipcode"].values[0]

combined_data.head()


In [20]:
# save the combined data to a csv file
combined_data.to_csv("citibike_combined_data.csv", index=False)